# 3 - Séparation de sources

In [ ]:
# Imports
from __future__ import division
from IPython.display import Audio, display
import numpy as np
import scipy as scp
import pylab as pyl
from matplotlib import cm
import matplotlib.pyplot as plt
import pywt
import holoviews as hv
hv.extension('bokeh')
import scipy.io as sio
from scipy import fftpack
from matplotlib.pyplot import imshow as imageplot
from mpl_toolkits.mplot3d import Axes3D
import wave
import warnings
from scipy.io.wavfile import read
import panel as pn
warnings.filterwarnings('ignore')
import logging

logging.getLogger("param").setLevel(logging.CRITICAL)

Le but de ce TP est de réaliser un programme python qui effectue une séparation de sources. Plus précisément un programme qui prend en entrée un son stéréophonique, mélange instantané de plusieurs sources sonores (ici des instruments de musique) et qui renvoie plusieurs sons stéréophoniques. On utilisera la transformée de Fourier à court terme (TFCT), ou transformée de Fourier à fenêtres, vue lors du TP précédent. 

On va utiliser le fait que, la plupart du temps, deux instruments ne jouent pas la même note au même moment et on va segmenter les spectrogrammes des deux canaux de manière à attribuer chaque atome temps-fréquence à un instrument, en fonction du rapport d'intensité entre les deux canaux, qui est supposé caractérisé chaque instrument.
Le TP se décompose en trois parties.

1) On considère un mélange de seulement deux instruments et on fait l'hypothèse que l'on connait les coefficients du mélange.

2) On considère un mélange de trois instruments et on fait l'hypothèse que l'on connait les coefficients du mélange.

3) On considère le même mélange de trois instruments mais cette fois ci on estime les coefficients de mélange.



Cette classe channel possède des fonctions qui permettent de manipuler le son. Notamment, celle qui permet d'avoir la transformée de fourier à court terme et celle permettant de faire la reconstruction du son

In [ ]:
# Définition des fonctions TFCourtTerme et RecSon
# On définit ces fonctions dans une classe Channel qui sera complétée au fur et à mesure du notebook

class Channel():

    # TFCourtTerme permet d'obtenir le tableau de transformée de Fourier à fenêtre d'un son
    # N représente la taille d'une fenêtre et rec le facteur de recouvrmenet des fenêtres
    @staticmethod
    def TFCourtTerme(Son,N:int=1024,rec:int=8):
        H=np.hanning(N)
        NS=len(Son)
        Nf=int(np.floor(rec*NS/N)-rec+1)
        TF=np.zeros((N,Nf),dtype=complex)
        for k in range(0,Nf):
            d=int(k*N/rec)
            f=d+N
            TF[:,k]=fftpack.fft(Son[d:f]*H)
        return TF


    # RecSon permet de ré-obtenir un vecteur représentant un son à partir d'un tableau de transformée de Fourier à fenêtre.
    # rec représente le facteur de recouvremnet utilisé lors de l'application de la transformée de Fourier à fenêtre
    @staticmethod
    def RecSon(TF,rec:int=8):
        N=np.shape(TF)[0]
        Nf=np.shape(TF)[1]
        H=np.hanning(N)
        Ns = int((N/rec)*(Nf+rec-0.5))
        Son= np.zeros(Ns,dtype=complex)
        for k in range(0,Nf):
            d=int(k*N/rec)
            f=d+N      
            Son[d:d+N]+=fftpack.ifft(TF[:,k])*H
        return Son

In [ ]:
# On définit une classe channel qui hérite d'elle-même pour pouvoir ajouter des méthodes
class Channel(Channel):
    def __init__(self, audio_path:str, sound_vector:np.array = None, channel_number:int=None):
        
        self.path = audio_path
        
        if audio_path is not None:
            self.vector = read(self.path)[1]
            self.sampling_frequency = read(self.path)[0]
        elif sound_vector is not None:
            self.vector = sound_vector
            self.sampling_frequency = 44100 # arbitrary value
        else:
            raise ValueError
        self.number = channel_number
        if self.number is not None:
            self.name = "channel {} ({})".format(self.number, self.path)
        else:
            self.name = "channel ({})".format(self.path)
            
    def __str__(self):
        return "File : {} (number = {})\n{}\nSampling frequency : {}".format(self.path, self.number, self.vector, self.sampling_frequency)
        
    def read(self, msg:str=None):
        if msg is None:
            print("Écoute de : {}".format(self.name))
        else:
            print(msg)
        display(Audio(self.vector, rate=self.sampling_frequency))
    
    def spectrogram(self, N:int = 1024, rec:int = 8, zoom:int = 100, width:int = 800, height:int = 300):        
        TF = self.TFCourtTerme(self.vector, N, rec)
        if zoom is not None:
            spectrum = np.abs(TF)[0:zoom,:]
        else:
            spectrum = np.abs(TF)
        return hv.Image(spectrum).opts(**dict(width=width,height=height,xaxis=None,yaxis=None,title=self.name))
    
# On définit une classe son nous aidant à manipuler et afficher les voix
class Son():
    def __init__(self, *args):
        self.channels = []
        counter = 1
        for channel_path in args:
            self.channels.append(Channel(channel_path, channel_number=counter))
            counter += 1

    def __str__(self):
        result = ""
        for channel in self.channels:
            result += channel.__str__() + "\n\n"
        return result
    
    def __getitem__(self, item_number):
        return self.channels[item_number-1]

    def spectrograms(self, N:int = 1024, rec:int = 8, zoom:int = 100):
        specs = []
        for channel in self.channels:
            specs.append(channel.spectrogram(N, rec, zoom, height=200))
        return pn.Column(*tuple(specs))

## Séparation de deux instruments

A l'aide la fonction read, nous avons chargé dans deux vecteurs différents $V1$ et $V2$ les sons Mix11.wav et Mix21.wav et afficher les tranformées de Fourier à fenêtres de chacune des deux voix (on utilisera TFCourtTerme).
On pourra afficher les basses fréquences pour mieux voir.
On fait l'hypothèse que les voix 1 et 2 (dites V1 et V2 ) est obtenue à partir des deux sources S1 et S2 de la manière suivante :
$$V_1=\frac{1}{3}S_1+\frac{2}{3}S_2\text{ et }V_2=\frac{2}{3}S_1+\frac{1}{3}S_2$$
Vous devriez voir les traces des deux instruments qui diffèrent par leur structure.

#### Chargement des deux Voies

In [ ]:
son = Son(r"./sons/Mix11.wav", r"./sons/Mix21.wav")

#### Affichage du spectogramme

In [ ]:
son.spectrograms(zoom=10)


<u>Commentaire</u> :
Les deux spectrogrammes nous permettent à priori de distinguer approximativement les deux instruments : la batterie, aux notes plus saccadées, et la guitare, dont les notes sont tenus plus longtemps. Partant de cette distinction, on remarque alors assez bien que l'intensité de la guitare est plus élevée dans le canal 1, celle de la batterie dans le canal 2.

#### Codage de la fonction de sépartion

Pour ce qui est de la fonction de sépatation, on regarde le rapport point par point des spectogrammes. Si ce rapport est supérieur à un certain seuil on attribue ce coefficient à l'instrument 1, sinon à l'instrument 2. On reconstruit ensuite les TFCT en utilisant les phases des voix 1 et 2.

In [ ]:
class Channel(Channel):
    
    def TF(self, N:int=1024, rec:int=8):
        return self.TFCourtTerme(self.vector,N,rec)
    
class Son(Son):
    
    @staticmethod
    def Separation2Instru(TFCT1, TFCT2, T):
        
        if TFCT1.shape != TFCT2.shape:
            raise ValueError('Parameters must have same size.')
            
        spec1 = np.abs(TFCT1)
        spec2 = np.abs(TFCT2)
        
        spec11 = np.zeros(spec1.shape)
        spec12 = np.zeros(spec1.shape)
        spec21 = np.zeros(spec2.shape)
        spec22 = np.zeros(spec2.shape)
        
        for row in range(spec1.shape[0]):
            for col in range(spec1.shape[1]):
                if spec1[row, col] >= T * spec2[row, col]:
                    spec11[row, col] = spec1[row, col]
                    spec21[row, col] = spec2[row, col]
                else:
                    spec12[row, col] = spec1[row, col]
                    spec22[row, col] = spec2[row, col]
        
        TFCT11 = spec11 * np.exp(1J * np.angle(TFCT1))
        TFCT12 = spec12 * np.exp(1J * np.angle(TFCT1))
        TFCT21 = spec21 * np.exp(1J * np.angle(TFCT2))
        TFCT22 = spec22 * np.exp(1J * np.angle(TFCT2))
        
        return TFCT11, TFCT12, TFCT21, TFCT22    
    
    def separation(self, T):
        TFCT11, TFCT12, TFCT21, TFCT22 = self.Separation2Instru(self[1].TF(),self[2].TF(), T)
        channel1 = Channel(None, Channel.RecSon(TFCT11))
        channel2 = Channel(None, Channel.RecSon(TFCT12))
        channel3 = Channel(None, Channel.RecSon(TFCT21))
        channel4 = Channel(None, Channel.RecSon(TFCT22))
        return channel1, channel2, channel3, channel4        

#### Choix du seuil 

Quel seuil peut-on proposer ici ? Proposer une formule générale en fonction des coefficients du mélange.

Pour déterminer le seuil, nous allons utiliser le fait qu'on connaisse les coefficients de mélanges des deux voies.
Supposons qu'on ait :
$$V_1=coef_{11}*S_1+coef_{12}*S_2\text{ et }V_2=coef_{21}*S_1+coef_{22}*S_2$$

Puisqu'on suppose que deux instruments ne jouent pas la même note au même moment (la plupart du temps), en effectuant le rapport terme à terme des coefficients des transformées de fouriers des deux voies, si on obtient un rapport égal à $r_1$ ($=\frac{coef_{11}}{coef_{21}}$), on affecte ce coefficent de fourier à la source S1, et s'il est égal à $r_2$ ($=\frac{coef_{12}}{coef_{22}}$) on affecte ce coefficent de fourier à la source S2. Ainsi, on peut réaliser une régle de décision pour affecter les coefficients à partir d'un seuil.


Le premier seuil auquel on pourrait penser est la moyenne arithmétique des rapports d'intensité $r_1$ et $r_2$. Cependant, la moyenne géométrique s'avère plus pertinente. En effet, on effectue la comparaison  de rapport d'intensité, ainsi en effectuant la moyenne géométrique le résultat obtenu est robuste par inversion si on tombe sur le cas d'un rapport $\frac{V1}{V2}$ ou $\frac{V2}{V1}$.Cela  vient du fait que la moyenne géométrique est invariante par l'inverse, ce qui n'est pas le cas pour la moyenne arithmétique.
Donc une formule générale en fonction des coefficients du mélange serait :
$$ seuil = \sqrt{\frac{coef_{11}}{coef_{21}} \times{} \frac{coef_{12}}{coef_{22}}}$$

En appliquant la formule précédente, on obtient dans notre cas :
$$
\text{seuil} = \sqrt{\frac{\frac{1}{3}}{\frac{2}{3}} \times{} \frac{\frac{2}{3}}{\frac{1}{3}}} = 1
$$  

In [ ]:
son = Son(r"./sons/Mix11.wav", r"./sons/Mix21.wav")
seuil = 1
channels = son.separation(seuil)

#### Test de la sépartion de source

Reconstruire ensuite à l'aide du programme RecSon, chacune des voix et écouter chacun des 4 sons produits. 
Commentez le résultat.

In [ ]:
son.channels[0].read()
son.channels[1].read()
channels[0].read(msg="Channel 1, source 1")
channels[1].read(msg="Channel 1, source 2")
channels[2].read(msg="Channel 2, source 1")
channels[3].read(msg="Channel 2, source 2")

<u>Commentaire</u> :

On obtient quatre sons, chacun représentant une source (deux sources dans notre cas) associée à une voie (deux channels dans notre cas). 

## Séparation de trois instruments

##### Chargement des deux voies avec un mélange de 3 instruments

Charger les nouvelles voix 1 et 2 à partir des fichiers Mix12.wav et Mix22.wav.
    On fait l'hypothèse que chacune des voix est maintenant un mélange instantané de trois instruments 
$$SV_1= \frac{2}{5}S_1 + \frac{1}{5}S_2 + \frac{4}{5}S_3\text{ et }V_2= \frac{3}{5}S_1 + \frac{4}{5}S_2 + \frac{1}{5}S_3$$


In [ ]:
son = Son(r"./sons/Mix12.wav", r"./sons/Mix22.wav")

#### Affichage des  spectogrammes des deux voies

Afficher les spectrogrammes des deux voix et visualiser les traces des trois instruments. 

In [ ]:
son.spectrograms(zoom=10)

#### Codage de la fonction de séparation de 3 instruments

Proposer un programme de séparation Separation3Instru qui prend en entrée deux TFCT, un vecteur $T$ seuil à 2 composantes et qui renvoie 6 TFCT associées aux deux voix de chacun des trois instruments. 



In [ ]:
class Son(Son):   
    
    @staticmethod
    def Separation3Instru(TFCT1, TFCT2, T1,T2):
        if TFCT1.shape != TFCT2.shape:
            raise ValueError('Parameters must have same size.')
        spec1 = np.abs(TFCT1)
        spec2 = np.abs(TFCT2)

        spec11 = np.zeros(spec1.shape)
        spec12 = np.zeros(spec1.shape)
        spec13 = np.zeros(spec1.shape)
        spec21 = np.zeros(spec2.shape)
        spec22 = np.zeros(spec2.shape)
        spec23 = np.zeros(spec2.shape)
        for row in range(spec1.shape[0]):
            for col in range(spec1.shape[1]):
                if spec1[row, col] >= T2 * spec2[row, col]:
                    spec13[row, col] = spec1[row, col]
                    spec23[row, col] = spec2[row, col]
                elif T1< (spec1[row, col]/spec2[row, col])< T2 :
                    spec11[row, col] = spec1[row, col]
                    spec21[row, col] = spec2[row, col]
                else :
                    spec12[row, col] = spec1[row, col]
                    spec22[row, col] = spec2[row, col]
        TFCT11 = spec11 * np.exp(1J * np.angle(TFCT1))
        TFCT12 = spec12 * np.exp(1J * np.angle(TFCT1))
        TFCT13 = spec13 * np.exp(1J * np.angle(TFCT1))
        TFCT21 = spec21 * np.exp(1J * np.angle(TFCT2))
        TFCT22 = spec22 * np.exp(1J * np.angle(TFCT2))
        TFCT23 = spec23 * np.exp(1J * np.angle(TFCT2))        

        return TFCT11, TFCT12, TFCT13,TFCT21, TFCT22,TFCT23
    
    def separation2(self,T1,T2) :      
        TFCT11, TFCT12, TFCT13,TFCT21, TFCT22,TFCT23 = self.Separation3Instru(self.channels[0].TF(),self.channels[1].TF(), T1,T2)
        channel1 = Channel(None, Channel.RecSon(TFCT11))
        channel2 = Channel(None, Channel.RecSon(TFCT12))
        channel3 = Channel(None, Channel.RecSon(TFCT13))
        channel4 = Channel(None, Channel.RecSon(TFCT21))
        channel5 = Channel(None, Channel.RecSon(TFCT22)) 
        channel6 = Channel(None, Channel.RecSon(TFCT23)) 
        return channel1, channel2, channel3, channel4, channel5, channel6 

##### Codage du programme qui calcul les seuils 

Ecrire un programme CalculSeuil qui prend en entrée 3 rapports d'intensité et qui renvoie deux seuils associés. 

In [ ]:
class Son(Son):

    @staticmethod
    def calculSeuil(r1,r2,r3):
        vr = np.array([r1,r2,r3])
        vr.sort()
        T2 = np.sqrt(vr[2]*vr[1])
        T1 = np.sqrt(vr[1]*vr[0])
        return T1,T2

#### Test du programme 

Tester le programme en utilisant les rapports d'intensités calculés à partir des coefficients de mélange, et commenter.
$$SV_1= \frac{2}{5}S_1 + \frac{1}{5}S_2 + \frac{4}{5}S_3\text{ et }V_2= \frac{3}{5}S_1 + \frac{4}{5}S_2 + \frac{1}{5}S_3$$

In [ ]:
class Son(Son):
    
    def separation3(self, r1, r2, r3):
        T1, T2 = self.calculSeuil(r1, r2, r3)
        return self.separation2(T1, T2)

In [ ]:
son = Son(r"./sons/Mix12.wav", r"./sons/Mix22.wav")
channels = son.separation3(2.0/3, 1.0/4, 4.0)
son.channels[0].read()
son.channels[1].read()
channels[0].read(msg="Channel 1, source 1")
channels[1].read(msg="Channel 1, source 2")
channels[2].read(msg="Channel 1, source 3")
channels[3].read(msg="Channel 2, source 1")
channels[4].read(msg="Channel 2, source 2")
channels[5].read(msg="Channel 2, source 3")

## Estimation des coeffients de mélange

On suppose dans cette partie qu'on veut effectuer la séparation précédente, sans connaitre les coefficients de mélange. On parle de séparation aveugle. On va donc chercher à estimer ces coefficients de mélange, ou plus précisément des rapports d'intensité des deux voix pour chacun des instruments. L'idée générale est simple : on va calculer un histogramme des rapports d'intensité des modules des TFCT et sélectionner les 3 valeurs les plus réprésentées dans l'histogramme, c'est à dire les troix maxima locaux de l'histogramme. Dans le détail on procède de la manière suivante :

1) On construit un tableau ModuleCarre qui va contenir la somme des carrés des modules des TFCT des deux voix.

2) On seuille ce tableau de manière à ne conserver que les N= 10% de coefficients les plus importants du tableau.

3) On construit une matrice $2\times N$ qui contient les pairs d'intensité associées des TFCT des deux voix.

4) On affiche sur un diagramme 2D les points obtenus (on ne les relie pas, on affiche par exemple des croix).

Les points obtenus doivent se répartir approximativement selon des nuages dirigés le long de droites dont les coefficients directeurs sont les rapports d'intensité.

5) On calcule un vecteur $R$ de rapports d'intensité (de longueur N).

6) On affiche un histogramme de $R$ et un histogramme de $U=\frac{R}{1+R}$. Vous devriez observer que les maxima de l'histogramme de $U$ sont plus marqués. 

7) Ecrire un programme qui estime les 3 valeurs des maximas de l'histogramme de U et donc les rapports $R_1$, $R_2$ et $R_3$ associés aux trois isntruments.

8) A l'aide du programme précédent, calculer les seuils optimaux et effectuer la séparation.

9) Commenter.

In [ ]:
def getBestCoef(TFCT1,TFCT2,plot=True):
        if TFCT1.shape != TFCT2.shape:
            raise ValueError('Parameters must have same size.')
        spec1 = np.abs(TFCT1)
        spec2 = np.abs(TFCT2)            
        # construction du tableau module carré
        moduleCarre =  np.abs(TFCT1)**2 + np.abs(TFCT2)**2
        mCarre_sorted = np.sort(np.reshape(moduleCarre,-1))[::-1]
        n_coef = len(mCarre_sorted)
        
        (N1,N2)=TFCT1.shape
        N = int(n_coef*0.1)
        seuil=mCarre_sorted[N-1]
        Pint=np.zeros((2,N))
        (N1,N2)=TFCT1.shape
        k=0
        for i in range(N1):
            for j in range(N2):
                if moduleCarre[i,j]>=seuil:
                    Pint[0,k]=spec1[i,j]
                    Pint[1,k]=spec2[i,j]
                    k+=1
        if plot :
            plt.scatter(Pint[0,:],Pint[1,:],s=2)
            plt.show()
        return Pint
        


In [ ]:
# 1) à 3)
TFCT1 =son.channels[0].TF()
TFCT2 =son.channels[1].TF()
spec1_carre = np.abs(TFCT1)**2
spec2_carre = np.abs(TFCT2)**2
moduleCarre =  spec1_carre + spec2_carre  
Pint = getBestCoef(TFCT1,TFCT2)

In [ ]:
def getRapportInt(Pint,plot=True,bins1=1000,bins2=1000) :
    R=Pint[0,:]/Pint[1,:]
    U = R/(1+R)
    if plot :
        plt.figure(0)
        plt.hist(R,bins=bins1)
        plt.show()
        plt.figure(1)
        plt.hist(U,bins = bins2)
        plt.show
    return R,U

In [ ]:
R,U  = getRapportInt(Pint,bins1=np.linspace(0,6,num=1000),bins2 = 1000)

In [ ]:
R,U  = getRapportInt(Pint,bins1=1000,bins2 = 1000)

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.mixture import GaussianMixture
def getMaximas(R):
    # Retrait des ouliers de notre ensemble R
    clf= IsolationForest(contamination=0.4)
    yhat = clf.fit_predict(R.reshape(-1, 1))
    mask = yhat != -1
    R_inliers = R[mask]
    # Calcul des maximas par Gaussians Mixture
    gm = GaussianMixture(n_components=3, random_state=0).fit(R_inliers.reshape(-1,1))
    moyennes = gm.means_[:,0]
    return moyennes

In [ ]:
moyennes = getMaximas(R)
print(moyennes)
print([r1,r2,r3])

In [ ]:
#moyennes = coeffs
T1,T2 = calculSeuil(moyennes[0],moyennes[1],moyennes[2])
sons_separes = separation2(son,T1,T2)

In [ ]:
sons_separes[0].read()

In [ ]:
sons_separes[1].read()

In [ ]:
sons_separes[2].read()

### Partie 5 : Séparation en utilisant des intervalles de confiances ou les valeurs précédentes

Dans cette partie, nous allons considérer le cas où certains instrument jouent en même temp la même note. Pour cela, on fait comme précédemment, c'est à dire on regarde le rapport point par point des spectogrammes. Mais on regarde aussi, à qui  a été attribué les n points précédents dans le spectre. Aprés avoir obtenu l'instrument, on lui attribue ce coefficient.

In [ ]:
from collections import Counter
def Separation3Pred(TFCT1, TFCT2, T1,T2,pred):
    if TFCT1.shape != TFCT2.shape:
        raise ValueError('Parameters must have same size.')
    spec1 = np.abs(TFCT1)
    spec2 = np.abs(TFCT2)
    pos = np.zeros(spec1.shape)
    spec11 = np.zeros(spec1.shape)
    spec12 = np.zeros(spec1.shape)
    spec13 = np.zeros(spec1.shape)
    spec21 = np.zeros(spec2.shape)
    spec22 = np.zeros(spec2.shape)
    spec23 = np.zeros(spec2.shape)
    for row in range(spec1.shape[0]):
        for col in range(spec1.shape[1]):
            try :
                if spec1[row, col] >= T2 * spec2[row, col]:
                    #print("je suis là")
                    pos[row,col] = 0
                    spec13[row, col] = spec1[row, col]
                    spec23[row, col] = spec2[row, col]
                    occurence = pos[row,col-pred:col]
                    if Counter(occurence).most_common(1) == 1 :
                        spec11[row, col] = spec1[row, col]
                        spec21[row, col] = spec2[row, col]
                    elif Counter(occurence).most_common(1) == 2:
                        spec12[row, col] = spec1[row, col]
                        spec22[row, col] = spec2[row, col]
                    else : 
                        None 
                elif T1< (spec1[row, col]/spec2[row, col])< T2 :
                    pos[row,col] = 1
                    spec13[row, col] = spec1[row, col]
                    spec23[row, col] = spec2[row, col]
                    occurence = pos[row,col-pred:col]
                    if Counter(occurence).most_common(1) == 0 :
                        spec11[row, col] = spec1[row, col]
                        spec21[row, col] = spec2[row, col]
                    elif Counter(occurence).most_common(1) == 2:
                        spec12[row, col] = spec1[row, col]
                        spec22[row, col] = spec2[row, col]
                    else : 
                        None 
                else :
                    pos[row,col] = 2
                    spec12[row, col] = spec1[row, col]
                    spec22[row, col] = spec2[row, col]
                    occurence = pos[row,col-pred:col]
                    if Counter(occurence).most_common(1) == 1 :
                        spec11[row, col] = spec1[row, col]
                        spec21[row, col] = spec2[row, col]
                    elif Counter(occurence).most_common(1) == 0:
                        spec13[row, col] = spec1[row, col]
                        spec23[row, col] = spec2[row, col]
                    else : 
                        None 
            except :
                if spec1[row, col] >= T2 * spec2[row, col]:
                    pos[row,col] = 0
                    spec13[row, col] = spec1[row, col]
                    spec23[row, col] = spec2[row, col]
                elif T1< (spec1[row, col]/spec2[row, col])< T2 :
                    pos[row,col] = 1
                    spec11[row, col] = spec1[row, col]
                    spec21[row, col] = spec2[row, col]
                else :
                    pos[row,col] = 2
                    spec12[row, col] = spec1[row, col]
                    spec22[row, col] = spec2[row, col]                
    TFCT11 = spec11 * np.exp(1J * np.angle(TFCT1))
    TFCT12 = spec12 * np.exp(1J * np.angle(TFCT1))
    TFCT13 = spec13 * np.exp(1J * np.angle(TFCT1))
    TFCT21 = spec21 * np.exp(1J * np.angle(TFCT2))
    TFCT22 = spec22 * np.exp(1J * np.angle(TFCT2))
    TFCT23 = spec23 * np.exp(1J * np.angle(TFCT2))        

    return TFCT11, TFCT12, TFCT13,TFCT21, TFCT22,TFCT23
def separation2(son,T1,T2,pred) :      
    TFCT11, TFCT12, TFCT13,TFCT21, TFCT22,TFCT23 = Separation3Pred(son.channels[0].TF(),son.channels[1].TF(), T1,T2,pred)
    channel1 = Channel(None, Channel.RecSon(TFCT11))
    channel2 = Channel(None, Channel.RecSon(TFCT12))
    channel3 = Channel(None, Channel.RecSon(TFCT13))
    channel4 = Channel(None, Channel.RecSon(TFCT21))
    channel5 = Channel(None, Channel.RecSon(TFCT22)) 
    channel6 = Channel(None, Channel.RecSon(TFCT23)) 
    return channel1, channel2, channel3, channel4,channel5,channel6 

In [ ]:
r1 =2/3
r2 =1/4
r3 =4.
T1,T2 = calculSeuil(r1,r2,r3)
pred = 0
sons_separes = separation2(son,T1,T2,pred)

In [ ]:
sons_separes[2].read()

Une autre façon de procéder est de considérer des intervalles de confiance. Au lieu de juste utiliser un seuil brute, on considère un intervalle de confiance. 

In [ ]:
def Separation3alpha(TFCT1, TFCT2, T1,T2,alpha1,alpha2,alpha3):
    if TFCT1.shape != TFCT2.shape:
        raise ValueError('Parameters must have same size.')
    spec1 = np.abs(TFCT1)
    spec2 = np.abs(TFCT2)

    spec11 = np.zeros(spec1.shape)
    spec12 = np.zeros(spec1.shape)
    spec13 = np.zeros(spec1.shape)
    spec21 = np.zeros(spec2.shape)
    spec22 = np.zeros(spec2.shape)
    spec23 = np.zeros(spec2.shape)
    for row in range(spec1.shape[0]):
        for col in range(spec1.shape[1]):
            if spec1[row, col] >= (T2+alpha1) * spec2[row, col]:
                spec13[row, col] = spec1[row, col]
                spec23[row, col] = spec2[row, col]
            elif (T1-alpha2)< (spec1[row, col]/spec2[row, col])< (T2+alpha2) :
                spec11[row, col] = spec1[row, col]
                spec21[row, col] = spec2[row, col]
            elif (T1+alpha3)>= (spec1[row, col]/spec2[row, col]):
                spec12[row, col] = spec1[row, col]
                spec22[row, col] = spec2[row, col]
    TFCT11 = spec11 * np.exp(1J * np.angle(TFCT1))
    TFCT12 = spec12 * np.exp(1J * np.angle(TFCT1))
    TFCT13 = spec13 * np.exp(1J * np.angle(TFCT1))
    TFCT21 = spec21 * np.exp(1J * np.angle(TFCT2))
    TFCT22 = spec22 * np.exp(1J * np.angle(TFCT2))
    TFCT23 = spec23 * np.exp(1J * np.angle(TFCT2))        

    return TFCT11, TFCT12, TFCT13,TFCT21, TFCT22,TFCT23
def separation2(son,T1,T2,alpha1,alpha2,alpha3) :      
    TFCT11, TFCT12, TFCT13,TFCT21, TFCT22,TFCT23 = Separation3alpha(son.channels[0].TF(),son.channels[1].TF(), T1,T2,alpha1,alpha2,alpha3)
    channel1 = Channel(None, Channel.RecSon(TFCT11))
    channel2 = Channel(None, Channel.RecSon(TFCT12))
    channel3 = Channel(None, Channel.RecSon(TFCT13))
    channel4 = Channel(None, Channel.RecSon(TFCT21))
    channel5 = Channel(None, Channel.RecSon(TFCT22)) 
    channel6 = Channel(None, Channel.RecSon(TFCT23)) 
    return channel1, channel2, channel3, channel4,channel5,channel6 

In [ ]:
r1 =2/3
r2 =1/4
r3 =4.
T1,T2 = calculSeuil(r1,r2,r3)
alpha1 = 0.01
alpha2 = 0.01
alpha3 = 0.01
sons_separes = separation2(son,T1,T2,alpha1,alpha2,alpha3)

In [ ]:
sons_separes[0].read()